In [2]:
from sentence_transformers import SentenceTransformer, LoggingHandler, InputExample
from sentence_transformers import models, util, datasets, evaluation, losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.cross_encoder import CrossEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import ndcg_score
from torch.utils.data import DataLoader
from rank_bm25 import BM25Okapi, BM25L, BM25Plus
from pyvi.ViTokenizer import tokenize
from tqdm import tqdm
import pandas as pd
import numpy as np
import json
import pickle

In [3]:
# Đọc dữ liệu test
with open("dvc_test.json", 'r') as file:
    test = json.load(file)

In [4]:
# Đọc dữ liệu các văn bản
vbpl = pd.read_csv('sent_truncated_vbpl_update.csv', encoding='utf-16')
vbpl = vbpl.dropna().reset_index(drop=True)
passages = vbpl['truncated_text'].to_numpy()

In [5]:
def word_segment(sent):
    sent = tokenize(sent.encode('utf-8').decode('utf-8'))
    return sent

In [6]:
def calculate_f2(precision, recall):        
    return (5 * precision * recall) / (4 * precision + recall + 1e-20)
def calculate_f1(precision, recall):        
    return (precision * recall) / (precision + recall + 1e-20)

In [8]:
with open("model/simcse-model-phobert-base-1x.pkl", "r") as file:
    bi_encoder = pickle.load(file)

In [9]:
with open("model/ce-model-05.pkl", "r") as file:
    cross_encoder = pickle.load(file)

In [10]:
# Load model
with open("model/bm25_plus", "rb") as bm_file:
    bm25 = pickle.load(bm_file)

In [11]:
import faiss
index = faiss.read_index("model/case_1x.index")

In [12]:
def evaluate(hits, k, scores):
    true_positive_set = set()
    false_positive_set = set()
    num_hits = 0
    average_precision = 0
    actual_relevance = []
      
    for j, idx_pred in enumerate(hits[:k].index, 1):
        key = (hits.at[idx_pred, "so_hieu"], hits.at[idx_pred, "dieu"])
        if key in dict_relevant:
            actual_relevance.append(1)
            true_positive_set.add(key)
            num_hits += 1
            average_precision += num_hits/j
        else:
            actual_relevance.append(0)
            false_positive_set.add(key)

    true_positive = len(true_positive_set)            
    false_positive = len(false_positive_set)
    
    if num_hits != 0: 
        average_precision = average_precision/num_hits
    
    ndcg = ndcg_score([actual_relevance], [scores[:k]], k=k)
    
    precision = true_positive/(true_positive + false_positive + 1e-20)
    recall = true_positive/actual_positive
    f1 = calculate_f1(precision, recall)
    f2 = calculate_f2(precision, recall)
    return precision, recall, f1, f2, average_precision, ndcg

In [13]:
# Đánh giá với dữ liệu
top_k = 500
top_k1 = 20
top_k2 = 5
thresh = 0
k1_total_f1 = 0
k1_total_f2 = 0
k1_total_precision = 0
k1_total_recall = 0
k1_total_map = 0
k1_total_ndcg = 0
k2_total_f1 = 0
k2_total_f2 = 0
k2_total_precision = 0
k2_total_recall = 0
k2_total_map = 0
k2_total_ndcg = 0
k3_total_f1 = 0
k3_total_f2 = 0
k3_total_precision = 0
k3_total_recall = 0
k3_total_map = 0
k3_total_ndcg = 0


for i, item in tqdm(enumerate(test)):
    query = item["noi_dung_hoi"]
    query_bm25 = query[:-1] if query.endswith("?") else query
    relevant_articles = item["vb_lien_quan"]
    dict_relevant = {(article["so_hieu"], article["dieu"]) : article for article in relevant_articles}
    actual_positive = len(relevant_articles)
    
    
    
    #### Block 1: Retrieve top k using BM25+ ####
    query_seg = word_segment(query_bm25)
    query_tokens = query_seg.split()
    doc_scores = bm25.get_scores(query_tokens)
    
    predictions = np.argpartition(doc_scores, len(doc_scores) - top_k)[-top_k:]
    hits_bm25 = vbpl.iloc[predictions]
    
    hits3 = hits_bm25.copy()
    hits3["bm25-score"] = doc_scores[:top_k] / (np.linalg.norm(doc_scores[:top_k])+1e-20)
    
    #### Rerank using bi-encoder top k1 ####
    embeddings = bi_encoder.encode(hits3['truncated_text'].to_list())
    query_embeddings = bi_encoder.encode([query])
    cos_scores = []
    for embedding in embeddings:
        cos_score = util.cos_sim(query_embeddings, embedding)
        cos_scores.append(cos_score.numpy()[0,0])
    hits3['cos-score'] = cos_scores
#     hits3['ensem-score'] = (1/2*hits3['cos-score']**2 + 1/2*hits3['bm25-score']**2)**1/2
    hits3['ensem-score'] = (hits3['cos-score'] * hits3['bm25-score'])**1/2
    
    hits3 = hits3.sort_values('ensem-score', ascending=False)
    
#     #### Rerank using cross-encoder ####  
#     cross_inp = [[query, passages[idx]] for idx in hits3[:top_k1].index]
#     cross_scores = cross_encoder.predict(cross_inp)
#     hits4 = hits3[:top_k1].copy()
#     hits4['cross-score'] = cross_scores

    
    
    #### Block 2: Retrieve top k using SimCSE ####
    query_embedding = bi_encoder.encode([query])
    normalized_query_embedding = query_embedding / np.linalg.norm(query_embedding)
    scores, indices, embeddings = index.search_and_reconstruct(normalized_query_embedding, top_k)
    hits_simcse = vbpl.iloc[indices[0]]
    hits1 = hits_simcse.copy()
    hits1['cos-score'] = scores[0]
    
    #### Rerank using BM25+ 2-grams top k1 ####
    corpus = []
    ngram_corpus2 = []
    for p in hits_simcse['truncated_text']:
        p = word_segment(p)
        p_tokens = p.split()
        corpus.append(p_tokens)
        ngram_corpus2 = [[ngram for ngram in zip(*[tokens[i:] for i in range(2)])] for tokens in corpus]
    
    bm25_rerank = BM25Plus(ngram_corpus2)
    
    query_seg = word_segment(query)
    query_tokens = query_seg.split()
    query_tokens_2 = [ngram for ngram in zip(*[query_tokens[i:] for i in range(2)])]
    bm25_scores = bm25_rerank.get_scores(query_tokens_2)
    hits1['bm25-score'] = bm25_scores / (np.linalg.norm(bm25_scores)+1e-20)
#     hits1['ensem-score'] = (1/2*hits1['bm25-score']**2 + 1/2*hits1['score']**2)**1/2
    hits1['ensem-score'] = (hits1['bm25-score']*hits1['cos-score'])**1/2
    hits1 = hits1.sort_values('ensem-score', ascending=False)
    
#     #### Rerank using cross-encoder ####    
#     cross_inp = [[query, passages[idx]] for idx in hits1[:top_k1].index]
#     cross_scores = cross_encoder.predict(cross_inp)
#     hits2 = hits1[:top_k1].copy()
#     hits2['cross-score'] = cross_scores
    
    
    #### Block 3: Ensemble Stage ####
    combine = pd.concat([hits1[:20], hits3[:20]])
    combine = combine.sort_values('ensem-score', ascending=False)
    hits_ensem = combine.drop_duplicates(subset=['so_hieu', 'dieu', 'truncated_text'], keep='first')
    
#     hits_ensem = hits_ensem.sort_values('ensem-score', ascending=False)
    
    precision, recall, f1, f2, average_precision, ndcg = evaluate(hits=hits_ensem, k=top_k1, scores=hits_ensem['ensem-score'].to_list())

    k1_total_precision += precision
    k1_total_recall += recall
    k1_total_f1 += f1
    k1_total_f2 += f2
    k1_total_map += average_precision
    k1_total_ndcg += ndcg
    
    precision, recall, f1, f2, average_precision, ndcg = evaluate(hits=hits_ensem, k=top_k2, scores=hits_ensem['ensem-score'].to_list())

    k2_total_precision += precision
    k2_total_recall += recall
    k2_total_f1 += f1
    k2_total_f2 += f2
    k2_total_map += average_precision
    k2_total_ndcg += ndcg
    
    precision, recall, f1, f2, average_precision, ndcg = evaluate(hits=hits_ensem, k=hits_ensem.shape[0], scores=hits_ensem['ensem-score'].to_list())

    k3_total_precision += precision
    k3_total_recall += recall
    k3_total_f1 += f1
    k3_total_f2 += f2
    k3_total_map += average_precision
    k3_total_ndcg += ndcg

547it [2:16:28, 14.97s/it]


In [14]:
N = len(test)
print(f"k = {top_k1}")
print(f"Recall: {k1_total_recall/N}")
print(f"Precision: {k1_total_precision/N}")
print(f"F2: {k1_total_f2/N}")
print(f"F1: {k1_total_f1/N}")
print(f"MAP: {k1_total_map/N}")
print(f"NDCG: {k1_total_ndcg/N}")

k = 20
Recall: 0.8634978671541742
Precision: 0.06480890732104808
F2: 0.24228212818558523
F1: 0.05956218072503938
MAP: 0.6286834254454082
NDCG: 0.7053359014830364


In [15]:
N = len(test)
print(f"k = {top_k2}")
print(f"Recall: {k2_total_recall/N}")
print(f"Precision: {k2_total_precision/N}")
print(f"F2: {k2_total_f2/N}")
print(f"F1: {k2_total_f1/N}")
print(f"MAP: {k2_total_map/N}")
print(f"NDCG: {k2_total_ndcg/N}")

k = 5
Recall: 0.7583790371724559
Precision: 0.224314442413163
F2: 0.49298741072049934
F1: 0.16721946548272038
MAP: 0.6456073532398938
NDCG: 0.6854849443011977


In [16]:
N = len(test)
print(f"k = {2*top_k1}")
print(f"Recall: {k3_total_recall/N}")
print(f"Precision: {k3_total_precision/N}")
print(f"F2: {k3_total_f2/N}")
print(f"F1: {k3_total_f1/N}")
print(f"MAP: {k3_total_map/N}")
print(f"NDCG: {k3_total_ndcg/N}")

k = 40
Recall: 0.8895490554539914
Precision: 0.04747139764060302
F2: 0.19018464250455538
F1: 0.04459440720959348
MAP: 0.6280865369375142
NDCG: 0.7109856557674793


In [ ]:
!huggingface-cli login

In [3]:
from transformers import pipeline
model_checkpoint = "nguyenvulebinh/vi-mrc-base"
nlp = pipeline('question-answering', model=model_checkpoint,
                   tokenizer=model_checkpoint)
QA_input = {
    'question': 'Hồ sơ của thủ tục cấp Giấy phép tổ chức thi người đẹp, người mẫu trong phạm vi địa phương gồm những gì? Thời gian thực hiện trong bao lâu?',
    'context': 'Điều 21 79/2012/NĐ-CP quy định về biểu diễn nghệ thuật, trình diễn thời trang; thi người đẹp và người mẫu; lưu hành, kinh doanh bản ghi âm, ghi hình ca múa nhạc, sân khấu Thủ tục cấp giấy phép tổ chức thi người đẹp, người mẫu 1. Tổ chức đề nghị cấp giấy phép tổ chức thi người đẹp, người mẫu gửi 01 bộ hồ sơ trực tiếp hoặc qua đường bưu điện, đến Cục Nghệ thuật biểu diễn hoặc Sở Văn hóa, Thể thao và Du lịch. Hồ sơ gồm: a) 01 đơn đề nghị cấp giấy phép tổ chức cuộc thi (Mẫu số 04); b) 01 đề án tổ chức cuộc thi, trong đó nêu rõ: Thể lệ cuộc thi, quy chế hoạt động của Ban tổ chức, Ban giám khảo; c) 01 văn bản đồng ý của Ủy ban nhân dân cấp tỉnh, nơi dự định tổ chức; d) 01 bản sao chứng thực hợp đồng hoặc văn bản thỏa thuận giữa tổ chức Việt Nam với tổ chức nước ngoài (bản dịch tiếng Việt có chứng nhận của công ty dịch thuật, đối với cuộc thi người đẹp, người mẫu quốc tế tổ chức tại Việt Nam). 2. Tổ chức nước ngoài tổ chức cuộc thi người đẹp, người mẫu quốc tế tại Việt Nam phải phối hợp với tổ chức Việt Nam có chức năng hoạt động văn hóa nghệ thuật. Tổ chức Việt Nam nộp hồ sơ theo quy định tại Khoản 1 Điều này. 3. Thời hạn cấp phép: Trong thời hạn 15 ngày làm việc (đối với cuộc thi người đẹp, người mẫu trong nước) và 30 ngày làm việc (đối với cuộc thi người đẹp, người mẫu quốc tế tổ chức tại Việt Nam), kể từ ngày nhận đủ hồ sơ hợp lệ, cơ quan nhà nước có thẩm quyền cấp giấy phép. Trường hợp không cấp giấy phép phải trả lời bằng văn bản và nêu rõ lý do.'
}
res = nlp(QA_input)
print('pipeline: {}'.format(res))

OSError: nguyenvulebinh/vi-mrc-base is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.